In [1]:
import pandas as pd

# =========================
# 설정
# =========================
INPUT_FILE = "lht65n08_converted.csv"
OUTPUT_FILE = "lht65n08_change_events.csv"

THRESHOLDS = {
    "battery_mv": 10.0,     # mV
    "temperature_c": 0.2,   # °C
    "humidity_pct": 1.0     # %
}

# =========================
# 로드 및 정렬
# =========================
df = pd.read_csv(INPUT_FILE, parse_dates=["time"])
df = df.sort_values("time").reset_index(drop=True)

# =========================
# 변화량 계산
# =========================
df["d_battery"] = df["battery_mv"].diff().abs()
df["d_temp"] = df["temperature_c"].diff().abs()
df["d_hum"] = df["humidity_pct"].diff().abs()

# =========================
# 의미 있는 변화만 필터
# =========================
change_df = df[
    (df["d_battery"] >= THRESHOLDS["battery_mv"]) |
    (df["d_temp"] >= THRESHOLDS["temperature_c"]) |
    (df["d_hum"] >= THRESHOLDS["humidity_pct"])
].copy()

# =========================
# 변화 간 시간 간격 (분)
# =========================
change_df["delta_t_min"] = (
    change_df["time"].diff().dt.total_seconds() / 60.0
)

# =========================
# 결과 저장
# =========================
change_df.to_csv(OUTPUT_FILE, index=False)

# =========================
# 요약 출력
# =========================
print("=== CHANGE EVENT SUMMARY ===")
print(f"Total raw samples      : {len(df)}")
print(f"Change events detected : {len(change_df)}")
print(f"Reduction ratio        : {len(change_df) / len(df):.4f}")

print("\n=== CHANGE CYCLE (minutes) ===")
print(change_df["delta_t_min"].describe(percentiles=[0.5, 0.75, 0.9]))

print(f"\nSaved to: {OUTPUT_FILE}")


=== CHANGE EVENT SUMMARY ===
Total raw samples      : 342689
Change events detected : 2072
Reduction ratio        : 0.0060

=== CHANGE CYCLE (minutes) ===
count    2071.000000
mean       83.607838
std       526.565598
min         0.100000
50%         3.000000
75%        15.000000
90%        71.983333
max      9635.016667
Name: delta_t_min, dtype: float64

Saved to: lht65n08_change_events.csv
